<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse Supervised
    - Entrainement Supervised
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 16/06/2020 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I - Sélection des datas entrantes</p>

In [1]:
def scoring(x_test, y_true, clf, lb, mode_supervise_with_lda = False, lda_model = None):
    nb_tag_1 = 0.0
    nb_tag_5 = 0.0
    classes = lb.classes_
    print(x_test.shape[0])
    no_top_words = 5
    for i in range(x_test.shape[0]):
        #for i in range(x_test.shape[0]):
        text_projection = x_test
        if(mode_supervise_with_lda):
            text_projection = lda_model.transform(x_test[i])
        '''
        print (text_projection)
        print (text_projection.shape)
        '''
        predicted = clf.predict_proba(text_projection)
        '''
        print (predicted)
        print (len(predicted))
        '''
        tempTag = [(1-item[0][0]) for item in predicted]
        list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
        '''
        tempTag = predicted[0]
        list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
        '''
        '''
        print (predicted)
        print (predicted.shape)
        '''
        list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
        #print (list_id_sorted)

        list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
        #print(list_id_sorted_suggested)
        prediction = [classes[id] for id in list_id_sorted_suggested]
        #print (str(prediction))


        l_y = [[i, x] for i, x in enumerate(y_true[i]) if x > 0]
        l_y_tagged = [x[0] for i, x in enumerate(l_y[:-no_top_words - 1:-1])]
        l_y_tags = [classes[id] for id in l_y_tagged]
        #print (l_y_tags)

        check_1 = False
        check_1 = any(item in prediction for item in l_y_tags)

        if check_1 is True:
            nb_tag_1 = nb_tag_1 + 1
        """    print("The list {} contains some elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """

        check_5 = False
        check_5 = all(item in prediction for item in l_y_tags)
        if check_5 is True:
            nb_tag_5 = nb_tag_5 + 1
        """    print("The list {} contains all elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        #str([tag for tag in y_true[i]]if tag ==1)
    return nb_tag_1, (100.0 * nb_tag_1 / float(x_test.shape[0])), nb_tag_5, (100.0 * nb_tag_5 / float(x_test.shape[0]))


In [2]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 
tagText = TagText()

tagText.nombre_post_entree = 5000
tagText.precision = 5000

print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)
data_question = tagText.read_source()
print(data_question.head())

data_preprocessed = data_question['body'].apply(tagText.preprocessing)

print(data_preprocessed.head())

C:\Users\naru_\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Data/
QuestionVsTags.csv
                                                body  \
0  <p>Here is a piece of C++ code that shows some...   
1  <p>I accidentally committed the wrong files to...   
2  <p>I want to delete a branch both locally and ...   
3  <p>What are the differences between <code>git ...   
4  <p>What is the use of the <code>yield</code> k...   

                                               title  \
0  Why is processing a sorted array faster than p...   
1  How do I undo the most recent local commits in...   
2  How do I delete a Git branch locally and remot...   
3  What is the difference between 'git pull' and ...   
4                  What does the "yield" keyword do?   

                                                tags  \
0  <java><c++><performance><optimization><branch-...   
1  <git><version-control><git-commit><undo><pre-c...   
2  <git><version-control><git-branch><git-push><g...   
3        <git><version-control><git-pull><git-fetch>   
4    <python><iterato

In [ ]:
def supervised_prepare_tagV2(self, data_preprocessed, data_tag):
    '''
    prepare classifier and class from file for supervised model
    '''
    from sklearn.pipeline import Pipeline
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer

    y_all = [
        item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
        for item in data_tag
    ]

    #print(y_train_tag)
    lb = tagText.MultiLabelBinarizer()
    Y_all = lb.fit_transform(y_all)


    # 80/20 split
    X_train, X_test, y_train, y_test = train_test_split(
        data_preprocessed, y_all, test_size=0.2,train_size=0.8, random_state=0)
    y_train = lb.transform(y_train)
    y_test = lb.transform(y_test)

    classifier = Pipeline([
        ('vectorizer', self.CountVectorizer()),
        ('tfidf', self.TfidfTransformer()),
        (
            'clf',
            self.RandomForestClassifier(
                n_estimators=100,
                max_depth=2,
                random_state=0
            )
        )
    ])  
    classifier.fit(X_train, y_train)
    
    print (len(X_test))
    print (len(y_test))
    print (y_test[0])
    print (X_test)
    
    nb1, score1, nb5, score5 = scoring(X_test, y_test, classifier, lb, False, None)
    
    print(score1)
    print(score5)
    
    return classifier, lb.classes_


In [ ]:
classifier, tags = supervised_prepare_tagV2(tagText, data_preprocessed, data_question['SelectedTags'])


1000
1000
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 

In [ ]:
classifier, tags = tagText.supervised_prepare_tagV2(data_preprocessed, data_question['SelectedTags'])



In [ ]:

message = 'git est un outils de code source. C\'est bien'
print(message)



print(tagText.supervised_predict(message, classifier, tags))

In [ ]:
print(tagText.supervised_predict(data_preprocessed[0], classifier, tags ))

In [ ]:
print (tags)

In [ ]:
tagText.supervised_predict('c c# .Net', classifier, tags )

In [ ]:
tagText.supervised_predict('how to convert int in flaot in c c# .Net', classifier, tags, 120000 )